In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('/Users/wangyingbo/Downloads/mnist',one_hot=True)
#每个批次的大小
batch_size = 100
#计算一共多少个批次
n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正太分布
    return tf.Variable(initial)

#初始化偏置
def biase_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#计算summary
def variable_summary(var):
    with tf.name_scope('summary'):
        mean = tf.reduce_mean(var) 
        tf.summary.scalar('mean',mean) #平均值
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(mean-var)))
    tf.summary.scalar('stddev',stddev) #标准差
    tf.summary.scalar('max',tf.reduce_max(var)) #最大值
    tf.summary.scalar('min',tf.reduce_min(var)) #最小值
    tf.summary.histogram('histogram',var)
#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#磁化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')

#改变x的格式，转为4D的向量[batch,in_height,in_width,in_chabbels]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
with tf.name_scope('layer1'):
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32]) #5*5的采样窗口，32个卷积核从1个平面抽取特征
        variable_summary(W_conv1)
    with tf.name_scope('b_conv1'):
        b_conv1 = biase_variable([32]) #每一个卷积核一个偏置值
        variable_summary(b_conv1)
    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) #进行max-pooling
    variable_summary(h_pool1)
#初始化第二个卷积层的权值和偏置
with tf.name_scope('layer2'):
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5,32,64]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
        variable_summary(W_conv2)
    with tf.name_scope('b_conv2'):
        b_conv2 = biase_variable([64]) #每一个卷积核一个偏置值
        variable_summary(b_conv2)
    #把h_pool1和权值向量进行卷积，再加上偏置值，然后用relu激活函数
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    variable_summary(h_pool2)
# 28*28的图片第一次卷积后还是28*28，第一次池化后变为14*14
# 第二次卷积后为14*14 第二次池化后变为7*7
#经过上面操作后得到64张7*7的平面

with tf.name_scope('fully_connect1'):
    #初始化第一个全连接层的权值
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64,1024]) #上一层有7*7*64个神经元，全连接层有1024个神经元
        variable_summary(W_fc1)
    with tf.name_scope('b_fc1'):
        b_fc1 = biase_variable([1024]) #1024个节点
        variable_summary(b_fc1)
#把池化层2的输出图片扁平化为1为
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

#keep_prob用来表示神经元的输出概率
rate = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,rate)
with tf.name_scope('fully_connect2'):
    with tf.name_scope('W_fc2'):
        # 初始化第二个全连接层
        W_fc2 = weight_variable([1024,10])                                                                          
        variable_summary(W_fc2)
    with tf.name_scope('b_fc2'):
        b_fc2 = biase_variable([10])
        variable_summary(b_fc2)
#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1,W_fc2) + b_fc2)
#交叉熵代价函数
with tf.name_scope('cross_entry'):
    cross_entropty = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropty)
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropty)
#结果存放在一个布尔列表
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) # argmax返回一维张量中最大的值所在的位置

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
tf.summary.scalar('accuracy',accuracy)
merged = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('cnn-logs/',sess.graph)
    for epoch in range(51):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys,rate:1.0})
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,rate:1.0})
        print('Iter '+ str(epoch) + ', Testing Accuracy = ' + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/wangyingbo/Downloads/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/wangyingbo/Downloads/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/wangyingbo/Downloads/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/wangyingbo/Downloads/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Iter 0, Testing Accura